In [1]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip


In [3]:
!mkdir -p /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles
!unzip /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles.zip -d /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles

Archive:  /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles.zip
  inflating: /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt  
  inflating: /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt  
  inflating: /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-07-spacex-starship-startups-future.txt  
  inflating: /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-03-checks-the-ai-powered-data-protection-project-incubated-in-area-120-officially-exits-to-google.txt  
  inflating: /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt  
  inflating: /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-04-microsoft-doubles-down-on-ai-with-new-bing-features.txt  
  inflating: /home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [10]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import openai
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

In [14]:
loader = DirectoryLoader("/home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles", glob="*.txt", loader_cls=TextLoader)

In [16]:
documents = loader.load()

In [18]:
documents

[Document(metadata={'source': '/home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-03-checks-the-ai-powered-data-protection-project-incubated-in-area-120-officially-exits-to-google.txt'}, page_content='After Google cut all but three of the projects at its in-house incubator Area 120 and shifted it to work on AI projects across Google, one of the legacy efforts — coincidentally also an AI project — is now officially exiting to Google. Checks, an AI-powered tool to check mobile apps for compliance with various privacy rules and regulations, is moving into Google proper as a privacy product aimed at mobile developers.\n\nChecks originally made its debut in February 2022, although it was in development for some time before that. In its time at Area 120, it became one of the largest projects in the group, co-founders Fergus Hurley and Nia Castelly told me, with 10 people fully dedicated to it and a number of others contributing less formally. The founders’ job titles under Google wi

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

In [23]:
chunk_text = text_splitter.split_documents(documents)

In [25]:
chunk_text[0]

Document(metadata={'source': '/home/muhammed-shafeeh/AI_ML/GenAI/vectordb/new_articles/05-03-checks-the-ai-powered-data-protection-project-incubated-in-area-120-officially-exits-to-google.txt'}, page_content='After Google cut all but three of the projects at its in-house incubator Area 120 and shifted it to work on AI projects across Google, one of the legacy efforts — coincidentally also an AI project — is now officially exiting to Google. Checks, an AI-powered tool to check mobile apps for compliance with various privacy rules and regulations, is moving into Google proper as a privacy product aimed at mobile developers.\n\nChecks originally made its debut in February 2022, although it was in development for some time before that. In its time at Area 120, it became one of the largest projects in the group, co-founders Fergus Hurley and Nia Castelly told me, with 10 people fully dedicated to it and a number of others contributing less formally. The founders’ job titles under Google wil

In [26]:
len(chunk_text)

227

In [27]:
from langchain import embeddings
persist_dir = 'db'

embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(chunk_text, embeddings, persist_directory=persist_dir)


/tmp/ipykernel_112259/3724987044.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [28]:
# persist the db to disk
vectordb.persist()
vectordb = None

/tmp/ipykernel_112259/2806010866.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [30]:
# load the db from disk
vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)

In [36]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [37]:
doc =retriever.get_relevant_documents("How much money did microsoft make in 2022?")

In [38]:
len(doc)

2

In [39]:
retriever.search_type

'similarity'

## **Make a Chain**

In [43]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [44]:
# example
query = "How much money did rise microsoft?"
result = qa_chain(query)
print(result['result'])

 I don't know, as the article does not mention a specific amount of money that Microsoft has invested. It only mentions that they have made a big investment, but does not provide a specific figure.


In [45]:
query = "what is the news about pando"
result = qa_chain(query)
print(result['result'])

 Pando has raised $30 million in a Series B funding round, bringing its total raised to $45 million. The new capital will be used to expand Pando's global sales, marketing, and delivery capabilities. The company is also open to exploring strategic partnerships and acquisitions.
